In [41]:
import requests
import pandas as pd
import time
import csv

ticker = "SPY"
# Define the base URL for the Alpha Vantage API
base_url = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&apikey="

# Replace YOUR_API_KEY with your Alpha Vantage API key
api_key = "MAEZUGVVHPARF6MJ"

# Initialize an empty dataframe to store the data
df = pd.DataFrame()
duration = 2 #alphavantage only offers 2 years of data with extended hours
timeframe = 1

"""
By default, adjusted=true and the output time series is adjusted by historical split and dividend events. 
Set adjusted=false to query raw (as-traded) intraday values.
When set to False:
2/3/2023 18:07	412.05	412.05	412.05	412.05	460
2/3/2023 18:05	412	412.1	412	412.1	5373
2/3/2023 18:04	412.05	412.05	412.05	412.05	167
2/3/2023 18:03	412.04	412.1	412	412.05	5242
2/3/2023 18:00	412	412	412	412	580
2/3/2023 17:59	412	412	412	412	1378
2/3/2023 17:58	412	412	412	412	514

When set to True:
2/3/2023 18:07	412.05	412.05	412.05	412.05	460
2/3/2023 18:05	412	412.1	412	412.1	5373
2/3/2023 18:04	412.05	412.05	412.05	412.05	167
2/3/2023 18:03	412.04	412.1	412	412.05	5242
2/3/2023 18:00	412	412	412	412	580
2/3/2023 17:59	412	412	412	412	1378
2/3/2023 17:58	412	412	412	412	514
"""
adjusted = True

# Loop over the number of years you want to download data for
for year in range(duration):
    # Define the start and end dates for the data slice
    y = year + 1
    
    for month in range(12):
        m = month + 1
        
        # Construct the URL for the data slice
        url = f"{base_url}{api_key}&symbol={ticker}&interval={timeframe}min&slice=year{y}month{m}&adjusted={adjusted}"
        print(url)
        # Make a request to the Alpha Vantage API
        response = requests.get(url)
        print(response)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Convert the response to a JSON object
            decoded_content = response.content.decode('utf-8')
            cr = csv.reader(decoded_content.splitlines(), delimiter=',')
            data = list(cr)
            print(len(data))
            print(data[0])
            print(data[1])

            # Convert the time series data to a pandas dataframe
            #year_df = pd.DataFrame(data, columns =['time', 'open', 'high', 'low', 'close', 'volume'])
            columns = data.pop(0)
            year_df = pd.DataFrame(data, columns = columns)
            year_df = year_df.astype({"open": float, "open": float, "high": float, "low": float, "close": float, "volume": int})
            
            year_df['time'] = pd.to_datetime(year_df['time'])
            year_df.set_index('time', inplace=True)
            print("index: ", year_df.index.name)

            # Convert the index to datetime
            #year_df.index = pd.to_datetime(year_df.index)
            
            print("downloaded count: ", year_df.count())
            print(year_df.head())

            # Append the data to the main dataframe
            df = df.append(year_df)
            print("total downloaded: ", df.count())

            # Wait for 21 seconds before making the next request to avoid exceeding the API rate limit 5 call per minute
            time.sleep(21)
        else:
            # If the request was not successful, print an error message
            print("Failed to retrieve data for year", "slice=year", y, "month", m, ". Status code:", response.status_code)

# Print the first 5 rows of the dataframe
print(df.head())
print(df.tail())

https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&apikey=MAEZUGVVHPARF6MJ&symbol=SPY&interval=1min&slice=year1month1&adjusted=True
<Response [200]>
18000
['time', 'open', 'high', 'low', 'close', 'volume']
['2023-02-03 20:00:00', '411.83', '411.83', '411.42', '411.5', '12037']
index:  time
downloaded count:  open      17999
high      17999
low       17999
close     17999
volume    17999
dtype: int64
                       open    high     low   close  volume
time                                                       
2023-02-03 20:00:00  411.83  411.83  411.42  411.50   12037
2023-02-03 19:59:00  411.85  411.85  411.85  411.85    1754
2023-02-03 19:58:00  411.85  411.85  411.84  411.85    3452
2023-02-03 19:57:00  411.88  411.88  411.88  411.88     285
2023-02-03 19:54:00  411.90  411.90  411.90  411.90     739
total downloaded:  open      17999
high      17999
low       17999
close     17999
volume    17999
dtype: int64
https://www.alphavantage.co/query?function=

https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&apikey=MAEZUGVVHPARF6MJ&symbol=SPY&interval=1min&slice=year1month9&adjusted=True
<Response [200]>
18161
['time', 'open', 'high', 'low', 'close', 'volume']
['2022-06-08 20:00:00', '406.25546193673654', '406.25546193673654', '406.25546193673654', '406.25546193673654', '379']
index:  time
downloaded count:  open      18160
high      18160
low       18160
close     18160
volume    18160
dtype: int64
                           open        high         low       close  volume
time                                                                       
2022-06-08 20:00:00  406.255462  406.255462  406.255462  406.255462     379
2022-06-08 19:59:00  406.156760  406.324553  406.156760  406.324553    1778
2022-06-08 19:58:00  406.206111  406.215981  406.146890  406.156760    8108
2022-06-08 19:57:00  406.186371  406.186371  406.156760  406.156760     265
2022-06-08 19:56:00  406.176501  406.206111  406.176501  406.206111    1

https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&apikey=MAEZUGVVHPARF6MJ&symbol=SPY&interval=1min&slice=year2month5&adjusted=True
<Response [200]>
17951
['time', 'open', 'high', 'low', 'close', 'volume']
['2021-10-11 20:00:00', '425.9427351100877', '425.9427351100877', '425.91331985248934', '425.93293002422155', '7868']
index:  time
downloaded count:  open      17950
high      17950
low       17950
close     17950
volume    17950
dtype: int64
                           open        high         low       close  volume
time                                                                       
2021-10-11 20:00:00  425.942735  425.942735  425.913320  425.932930    7868
2021-10-11 19:59:00  425.972150  425.981955  425.952540  425.972150    1694
2021-10-11 19:58:00  425.923125  425.972150  425.893710  425.952540    2871
2021-10-11 19:57:00  425.834879  425.913320  425.834879  425.913320    2897
2021-10-11 19:56:00  425.805464  425.825074  425.805464  425.825074    25

                       open    high     low   close  volume
time                                                       
2023-02-03 20:00:00  411.83  411.83  411.42  411.50   12037
2023-02-03 19:59:00  411.85  411.85  411.85  411.85    1754
2023-02-03 19:58:00  411.85  411.85  411.84  411.85    3452
2023-02-03 19:57:00  411.88  411.88  411.88  411.88     285
2023-02-03 19:54:00  411.90  411.90  411.90  411.90     739
                           open        high         low       close  volume
time                                                                       
2021-02-16 04:05:00  383.236135  383.245844  383.236135  383.236135     808
2021-02-16 04:04:00  383.236135  383.236135  383.236135  383.236135     205
2021-02-16 04:03:00  383.255554  383.274972  383.226426  383.226426    1675
2021-02-16 04:02:00  383.255554  383.313810  383.255554  383.284682    1020
2021-02-16 04:01:00  382.935147  383.284682  382.935147  383.284682    1202


In [49]:
import os

path = 'E:\Day Trading\Historical Data\SPY\Alphavantage'
file = "SPY_2021-02-16_TO_2023-02-03.csv"
output_file = os.path.join(path, file)
export_csv = df.to_csv(output_file, header=True, index=True)